<a href="https://colab.research.google.com/github/artluiz/Geoespaciais-Lista-4/blob/main/5967_Prova2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# o geopandas não está instalado no Colab por default
!pip install --upgrade geopandas

In [ ]:
# dados e geodados
import pandas as pd
import geopandas as gpd

# download de arquivos
import urllib.request

#visualização
import matplotlib.pyplot as plt

import folium
import numpy

%matplotlib inline

# Questão 1

## Carga dos dados

In [ ]:
url= 'https://geoservicos.pbh.gov.br/geoserver/wfs?service=WFS&version=1.0.0&request=GetFeature&typeName=ide_bhgeo:BAIRRO_OFICIAL&srsName=EPSG:31983&outputFormat=application%2Fjson'
bairros = gpd.read_file(url)
bairros.head()

In [ ]:
url= 'https://geoservicos.pbh.gov.br/geoserver/wfs?service=WFS&version=1.0.0&request=GetFeature&typeName=ide_bhgeo:ESCOLAS_MUNICIPAIS_ENSINO_FUNDAMENTAL&srsName=EPSG:31983&outputFormat=application%2Fjson'
escolas = gpd.read_file(url)
escolas = escolas[['ID_EQUIP_EDUCACAO', 'NOME','CODIGO_INEP', 'BAIRRO', 'geometry']]
escolas.head()

In [ ]:
num_join_espacial = bairros.sjoin(escolas, how='inner', predicate='contains').shape[0]
num_join_atributos = bairros.merge(escolas, left_on='NOME', right_on='BAIRRO', how='inner').shape[0]

print('Registros retornados')
print('......join espacial: ', num_join_espacial)
print('.join por atributos: ', num_join_atributos)

## Resposta da Questão 1

In [ ]:
# Resposta da questão 1
#Faz o join espacial nas duas tabelas com o método right para manter a ordenação da tabela escolas
join_espacial = bairros.sjoin(escolas, how='right', predicate='contains')
#copia escolas para o novo aruivo new_escolas
new_escolas = escolas
#faz a coluna bairro receber a NOME_left que possui os nomes dos bairros de cada escola após o join e respeita a ordenação do gbd escolas, assim eles podem apenas serem trocados
new_escolas['BAIRRO'] = join_espacial['NOME_left']
new_escolas

# Questão 2

## Resposta da Questão 2

In [ ]:
# Calcula a quantidade de escolas por km² em cada bairro ao iterar sobre cada geom de bairro e usando sum() para ter a quantidade total de escolas por bairro
# e então dividir com a área ao uadrado de cada bairro
bairros['escolas_km2'] = bairros['geometry'].apply(lambda geom: escolas.within(geom).sum() / (geom.area / 1000000))

# Define a quantidade de figuras que terá no subplot mais o tamanho delas
fig, ax = plt.subplots(1, 1, figsize=(12, 8))

# Especifica a tabela qual será plotada e define a variável do mapa coroplético e os detalhes do mapa, como as linhas que representam as divisas dos bairros e sistema de cores
bairros.plot(column='escolas_km2', cmap='gist_earth', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)

# Define o título do mapa
plt.title('Quantidade de Escolas por Metro Quadrado')
plt.show()

# Questão 3

### Carga dos dados

In [ ]:
# Links para download dos dados
urlbase = 'https://drive.google.com/uc?export=download&id='

rios_mg        = gpd.read_file(urlbase + '16V3_wSqo8FQRKnv15bxmQiJgKMeRc8Y2')
estado_mg      = gpd.read_file(urlbase + '10380Yq-sOZUDJ4Ez2_2hiIdMfgsAqUEH')
rodovias_mg    = gpd.read_file(urlbase + '18yYnsdpkuFZRJG0WmBe1CKyojLvaCE4g')

## Resposta da Questão 3

In [ ]:
# Resposta da questão 3

# Verifica as rodovias pavimentadas ao selecionar na tabela rodovias_mg apenas as que apresentem o tipo Pavimentada
rodovias_pavimentadas = rodovias_mg[rodovias_mg['tipopnv'] == 'Pavimentada']

# Realiza a sobreposição dos geom de ambos gbd com o parâmetro intersection para que seja mantido apenas as partes que se intersectam
pontes_mg = gpd.overlay(rodovias_pavimentadas, rios_mg, how='intersection', keep_geom_type=False)

# Remover a coluna geometry anterior, que após a sobreposição não é mais necessária
pontes_mg = pontes_mg[['geometry']]

# Visualização do GeoDataFrame gerado com possíveis localizações de pontes
print(pontes_mg)

#Quantidade total de possíveis pontes
text = 'Quantidade total de possíveis pontes: '
num = pontes_mg.shape[0] + 1
print(text + str(num))

# Questão 4

## Resposta da Questão 4

In [ ]:
# Resposta da questão 4
# Criação de uma figura vazia com tamaho bom para a vizualição do GeoDataFrame
fig, ax = plt.subplots(figsize=(10, 10))

# Plotagem das camadas referentes as rodovias, rios e possíveis localizações de pontes
rodovias_pavimentadas.plot(ax=ax, color='black', linewidth=1, label='Rodovias Pavimentadas')
rios_mg.plot(ax=ax, color='purple', linewidth=1, label='Rios')
pontes_mg.plot(ax=ax, color='yellow', linewidth=0.5, label='Possíveis Pontes')

# Ajuste dos limites do mapa para melhor visualização
ax.set_xlim(estado_mg.total_bounds[0], estado_mg.total_bounds[2])
ax.set_ylim(estado_mg.total_bounds[1], estado_mg.total_bounds[3])

# Título do ampa
ax.set_title('Mapa de Rios, Rodovias e Pontes em Minas Gerais')

# Adicionando legenda para melhor compreenção
ax.legend()

# Exibição da figura gerada
plt.show()